In [ ]:
import numpy as np
import pandas as pd
import math
import polars as pl
import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.backtest import Backtest
from strategy.data import PoolDataUniV3, RawDataUniV3, SyntheticData
from strategy.portfolio import Portfolio
from strategy.positions import UniV3Position, BiCurrencyPosition
from strategy.viewers import PotrfolioViewer, RebalanceViewer, UniswapViewer, LiquidityViewer
from strategy.strategies import AbstractStrategy, HStrategy, UniV3Passive, MStrategy
from strategy.multi_strategy import MultiStrategy
from strategy.uniswap_utils import UniswapLiquidityAligner, UniswapV2Utils, UniswapV3Utils
from strategy.primitives import Pool, Token, Fee

In [ ]:
def tick_to_price(tick, decimal_diff=10):
    price = np.power(1.0001, tick) / 10 ** decimal_diff
    return price

def price_to_tick(price, decimal_diff=10):
    tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
    return int(round(tick))

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)

In [ ]:
data = RawDataUniV3(pool).load_from_folder()

In [ ]:
data.swaps.head()

## UniswapV2 strategy

In [ ]:
univ3_passive = UniV3Passive(1e-16, 1e16, pool, 0.01)

In [ ]:
b = Backtest(univ3_passive)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
df1 = portfolio_history.calculate_stats()

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6 = PotrfolioViewer(portfolio_history, pool).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

## H-strategy (0 - inf)

In [ ]:
h_strat = HStrategy(800, 10, 1200, 1e-16, 1e16, 60, 15, pool, 0.01, 1e-5, 1e-5)

In [ ]:
b = Backtest(h_strat)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
df2 = portfolio_history.calculate_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6 = PotrfolioViewer(portfolio_history, pool).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

## H-Strategy concentrated (with uniswap)

In [ ]:
m_strat = HStrategy(800, 10, 1200, 10, 20, 60, 15, pool, 0.01, 1e-5, 1e-5)

In [ ]:
b = Backtest(m_strat)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
df3 = portfolio_history.calculate_stats()

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6 = PotrfolioViewer(portfolio_history, pool).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

# M-Strategy 

In [ ]:
m_strat = MStrategy(800, 10, 20, pool, 0.01, 1e-5, 1e-5)

In [ ]:
b = Backtest(m_strat)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
df4 = portfolio_history.calculate_stats()

In [ ]:
rv = RebalanceViewer(rebalance_history)
rebalanses = rv.draw_rebalances(data.swaps)

In [ ]:
rebalanses

In [ ]:
fig1, fig2, fig3, fig4, fig5, fig6 = PotrfolioViewer(portfolio_history, pool, 7).draw_portfolio()

In [ ]:
fig1

In [ ]:
fig2

In [ ]:
fig3

In [ ]:
fig4

# Aggregate results

In [ ]:
def aggregate_column(col, offset, **kwargs):
    fig = go.Figure()
    
    for name, df in kwargs.items():
        delta = datetime.timedelta(days=offset)
        start_date = df['timestamp'][0] + delta
        df2 = df.filter(pl.col('timestamp') >= start_date)
         
        fig.add_trace(
            go.Scatter(
                x=df2['timestamp'].to_list(),
                y=df2[col],
                name=name,
            ))
            
    fig.update_xaxes(title_text="Timeline")
    return fig

In [ ]:
data_aggr = {'UniswapV2': df1, "H strategy 0-inf": df2, "H strategy min-max": df3, 'M strategy': df4}

In [ ]:
fig = aggregate_column('vpn_apy_y', 30, **data_aggr)
fig.update_yaxes(title_text="APY in ETH")
fig.update_layout(title='Strategies APY p_last comparison in ETH. WBTC/ETH pool.', yaxis_range=[0, 100])

In [ ]:
fig = aggregate_column('vp0_apy_y', 30, **data_aggr)
fig.update_yaxes(title_text="APY in ETH")
fig.update_layout(title='Strategies APY p_first comparison in ETH. WBTC/ETH pool.', yaxis_range=[0, 100])

In [ ]:
fig = aggregate_column('portfolio_apy_y', 30, **data_aggr)
fig.update_yaxes(title_text="APY in ETH")
fig.update_layout(title='Strategies APY comparison in ETH. WBTC/ETH pool.', yaxis_range=[-50, 100])

In [ ]:
labels = ['UniswapV2', "H strategy 0-inf", "H strategy min-max", 'M strategy']

In [ ]:
# apy_x_1 = 100*(df1['hold_apy_x'][-1])
apy_x_2 = (df1['vpn_apy_x'][-1])
apy_x_3 = (df2['vpn_apy_x'][-1])
apy_x_4 = (df3['vpn_apy_x'][-1])
apy_x_5 = (df4['vpn_apy_x'][-1])


In [ ]:
# apy_y_1 = 100*(df1['hold_apy_y'][-1])
apy_y_2 = (df1['vpn_apy_y'][-1])
apy_y_3 = (df2['vpn_apy_y'][-1])
apy_y_4 = (df3['vpn_apy_y'][-1])
apy_y_5 = (df4['vpn_apy_y'][-1])


In [ ]:
apys_x = [apy_x_2, apy_x_3, apy_x_4, apy_x_5]

In [ ]:
apys_y = [apy_y_2, apy_y_3, apy_y_4, apy_y_5]

In [ ]:
fig = px.bar(y=apys_x, x=labels)
fig.update_xaxes(title_text="Strategy")
fig.update_yaxes(title_text="APY", secondary_y=False)
fig.update_layout(title='Strategies APY comparison in WBTC. WBTC/ETH pool.')

In [ ]:
fig = px.bar(y=apys_y, x=labels)
fig.update_xaxes(title_text="Strategy")
fig.update_yaxes(title_text="APY", secondary_y=False)
fig.update_layout(title='Strategies APY comparison in ETH. WBTC/ETH pool.')